In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io
import scanpy as sc
import tacco as tc

In [2]:
save = False
dirname = '../datasets/'
outdir = '../output/'

# Fig4 - mapping scRNA-seq onto Slide-seq

Projecting high-quality fibroblasts expression data from Muhl et al. onto spatially-informed Slide-seq data (Avraham-Davidi et al.). 


## Standardize the datasets

### Construct "scdata" for Muhl single-cell data

In [3]:
# read expression and tasks (from Miri)

dirname = '../datasets/'

# read genenames
scgenenames = pd.read_csv(os.path.join(dirname, 'colonSCGeneNames.txt'), header=None, index_col=0).index

# read computed task allocations
sctasks = pd.read_csv(os.path.join(dirname, 'colonSCFibsArcsCoef.csv'), header=None)

# read expression
scdata_org = pd.read_csv(os.path.join(dirname, 'GSE149859_colon_processed_counts.txt.gz'), sep='\t')

print(f"Muhl single-cell data, ngenes: {scdata_org.shape[0]}, ncells: {scdata_org.shape[1]}")
                         

Muhl single-cell data, ngenes: 46609, ncells: 1646


In [4]:
# convert gene id to gene names

# read id-name table
genenames_df = pd.read_csv(os.path.join(dirname, 'colonSCGenesConvert.txt'), sep='\t')
genenames_df['Name'] = genenames_df['Description'].apply(lambda s: s.split('(')[-1].split(')')[0])

gene_id_to_name = genenames_df[['Name']]
gene_id_to_name.index = genenames_df['ENSEMBL_GENE_ID']

genes_with_names = list(set(scdata_org.index).intersection(gene_id_to_name.index))

In [5]:
# subset to genes with names and replace id with name
scdata_ns = scdata_org.loc[genes_with_names].copy()
ids = gene_id_to_name.loc[genes_with_names].index
scdata_ns.index = gene_id_to_name.loc[ids]['Name'].values
# scdata_ns.index = gene_id_to_name.loc[genes_with_names].loc[~ids.duplicated(keep='first')]['Name'].values
scdata_ns = scdata_ns.T


In [6]:
# some gene names have multiple ids, dropping these because can't resolve in ss data
genenames_df.index = genenames_df['Name']
genenames_uniq = genenames_df['Name'].drop_duplicates(keep=False).index
genenames_df = genenames_df.loc[genenames_uniq]
genenames_mapped = list(set(scgenenames.values).intersection(set(genenames_df.index.values)))
geneids_mapped = genenames_df.loc[genenames_mapped]['ENSEMBL_GENE_ID'] 

In [8]:
# construct single cell data AnnData object -- scdata
df = scdata_org.loc[geneids_mapped]                              
df.index = genenames_mapped
scdata = sc.AnnData(df.T)

# add tasks
sctasks.index = df.columns 
sctasks.columns = ['task %d' % (c + 1) for c in sctasks.columns]
scdata.obs = pd.concat((scdata.obs, sctasks), 1)
scdata.obsm['tasks'] = sctasks

print(f"Muhl single-cell data subset to genes with names, ngenes: {scdata.shape[1]}, ncells: {scdata.shape[0]}")

Muhl single-cell data subset to genes with names, ngenes: 8425, ncells: 1646


/var/folders/8y/cpqrd46n7ml80f139vjz9l240000gn/T/ipykernel_45670/3592150699.py:4: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  scdata = sc.AnnData(df.T)
/var/folders/8y/cpqrd46n7ml80f139vjz9l240000gn/T/ipykernel_45670/3592150699.py:9: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  scdata.obs = pd.concat((scdata.obs, sctasks), 1)


### Construct "ssdata" for Slide-seq data

In [9]:
# read slideseq
puck = '20'

ssdata_loc = pd.read_csv(os.path.join(dirname, f'2020-09-14_Puck_200701_{puck}_fibro_thr0.5_loc.csv'), index_col=0)
ssdata_ct = pd.read_csv(os.path.join(dirname, f'2020-09-14_Puck_200701_{puck}_fibro_thr0.5_ct.csv'), index_col=0)
ssdata_exp = pd.read_csv(os.path.join(dirname, f'2020-09-14_Puck_200701_{puck}_fibro_thr0.5_exp.csv'), index_col=0)

print(f"Slide-seq data, ngenes: {ssdata_exp.shape[1]}, nbeads: {ssdata_exp.shape[0]}")

Slide-seq data, ngenes: 19626, nbeads: 2559


In [10]:
# read sc tasks
ssdata_task = pd.read_csv(os.path.join(dirname, f'colonFibP{puck}TaskPos.csv'), header=None)
ssdata_task.index = ssdata_exp.index
ssdata_task.columns = ['task %d' % (c + 1) for c in ssdata_task.columns]

### Subset to scdata and ssdata shared genes

In [11]:
# subseting for genenames to match
genenames = list(set(ssdata_exp.columns).intersection(genenames_mapped))
ssdata = sc.AnnData(ssdata_exp[genenames], obs=ssdata_loc)
scdata = scdata[:, genenames]
ssdata.obsm['tasks'] = ssdata_task

print(f"Single-cell data subset to shared genes, ngenes: {scdata.shape[1]}, ncells: {scdata.shape[0]}")
print(f"Slide-seq data subset to shared genes, ngenes: {ssdata.shape[1]}, nbeads: {ssdata.shape[0]}")


Single-cell data subset to shared genes, ngenes: 8190, ncells: 1646
Slide-seq data subset to shared genes, ngenes: 8190, nbeads: 2559


/var/folders/8y/cpqrd46n7ml80f139vjz9l240000gn/T/ipykernel_45670/3786744548.py:3: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  ssdata = sc.AnnData(ssdata_exp[genenames], obs=ssdata_loc)


## Map ssdata using scdata as reference

In [13]:
annotation_key = 'cell_id' # this holds the "labels" of the reference data, can be cell types, states, here, it is the cells' ids
result_key = 'cell_id_mapping' # will hold the resulting mapping in the adata object

scdata.obs[annotation_key] = scdata.obs_names
# tc.tl.annotate(ssdata, scdata, annotation_key=annotation_key, result_key=result_key)
tc.tl.annotate(ssdata, scdata, annotation_key=annotation_key, result_key=result_key, platform_iterations=1, bisections=0, bisection_divisor=1)


Starting preprocessing
Finished preprocessing in 0.93 seconds.
Starting annotation of data with shape (2559, 7885) and a reference of shape (1646, 7885) using the following wrapped method:
+- platform normalization: platform_iterations=1, gene_keys=cell_id, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  0.0005081116214323467 0.0017364441299349783
scale.....time 156.71984720230103
mean,std( rescaling(gene) )  0.9700986626537735 0.15339693069007093
Finished annotation in 176.1 seconds.


AnnData object with n_obs × n_vars = 2559 × 8190
    obs: 'x', 'y'
    obsm: 'tasks', 'cell_id_mapping'
    varm: 'cell_id_mapping'

## Predict tasks and expression based on mapping

In [18]:
# predicted tasks
mapping = ssdata.obsm[result_key].T # sums to 1 per bead
pred_tasks = (scdata.obsm['tasks'].T @ mapping).T
pred_tasks.index = ssdata.obs_names
ssdata.obsm['pred tasks'] = pred_tasks

pt_col = 'pred tasks'
# save just tasks
if save:
    ssdata.obsm['pred tasks'].to_csv(os.path.join(outdir, f'sc_on_ss_pred_tasks.csv'))
    ssdata.obs[['x', 'y']].to_csv(os.path.join(outdir, f'sc_on_ss_pred_loc.csv'))


# predicted expression
pred_exp_all = (scdata_ns.T @ mapping).T
pred_exp_all.index = ssdata.obs_names

# save all
sspred_adata = sc.AnnData(pred_exp_all, obs=ssdata_loc)
sspred_adata.obsm['tasks'] = ssdata.obsm['pred tasks']

if save:
    sc.write(os.path.join(outdir, f'sc_on_ss_pred_adata_{puck}.h5'), sspred_adata)

/var/folders/8y/cpqrd46n7ml80f139vjz9l240000gn/T/ipykernel_45670/827273562.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  sspred_adata = sc.AnnData(pred_exp_all, obs=ssdata_loc)
/Users/nomo/opt/miniconda3/envs/pareto_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
